<a href="https://colab.research.google.com/github/thisisZAQ/Decoding-Fiscal-Narratives-A-20-Year-NLP-Study-of-India-s-Union-Budgets/blob/main/Budget_Speeches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gettng Started**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# Load the CSV file
file_path = '/content/Speech_20years.csv'
df = pd.read_csv(file_path)

# Clean the column names by stripping extra spaces
df.columns = df.columns.str.strip()

# Drop rows with missing speech data and group by Year and Finance Minister
df_cleaned = df[['Year', 'Minister', 'Speech']].dropna()
df_grouped = df_cleaned.groupby(['Year', 'Minister'])['Speech'].apply(lambda x: ' '.join(x)).reset_index()



In [ ]:
# Calculate the length of each speech in words
df_grouped['Speech Length'] = df_grouped['Speech'].apply(lambda x: len(x.split()))

# Group by Finance Minister and calculate the total speech length for each minister
speech_length_by_minister = df_grouped.groupby('Minister')['Speech Length'].sum().reset_index()

# Sort by speech length for better visualization
speech_length_by_minister = speech_length_by_minister.sort_values(by='Speech Length', ascending=False)

# Display the results
speech_length_by_minister.head()


In [ ]:
# Calculate the length of each speech in words
df_grouped['Speech Length'] = df_grouped['Speech'].apply(lambda x: len(x.split()))

# Group by Finance Minister and calculate the total speech length for each minister
speech_length_by_minister = df_grouped.groupby('Minister')['Speech Length'].sum().reset_index()

# Sort by speech length for better visualization
speech_length_by_minister = speech_length_by_minister.sort_values(by='Speech Length', ascending=False)

# Display the results
speech_length_by_minister.head()

df_grouped.tail()
speech_length_by_minister.head()


In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='Speech Length', data=df_grouped, marker='o')
plt.title('Speech Length Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Words')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import plotly.express as px

# Create the bar plot using Plotly Express
fig = px.bar(speech_length_by_minister,
             x='Speech Length',
             y='Minister',
             orientation='h',  # Horizontal bar plot
             title='Total Speech Length by Finance Minister',
             labels={'Speech Length': 'Speech Length (Word Count)', 'Minister': 'Minister'},
             color='Speech Length',  # Color based on speech length
            template='plotly_dark',
             color_continuous_scale='Viridis')

# Customize layout (optional)
fig.update_layout(
    xaxis_title="Total Speech Length (Word Count)",
    yaxis_title="Finance Minister",
    height=600,
    width=900
)

# Show the plot
fig.show()



In [ ]:
%pip install wordcloud

In [ ]:
from wordcloud import WordCloud

# Combine all speeches into one large text
all_text = ' '.join(df_grouped['Speech'])

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import spacy
from spacy import displacy

# Load the spaCy language model
nlp = spacy.load("en_core_web_sm")

# Visualize named entities for the first speech
doc = nlp(df_grouped['Speech'][3])
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# Define themes and associated keywords
themes = {
    'inflation': ['inflation', 'prices', 'cost of living'],
    'tax': ['tax', 'taxation', 'income tax', 'corporate tax','tax bracket','tax slabs'],
    'growth': ['growth', 'development', 'GDP', 'economy','globalisation'],
    'employment': ['employment', 'jobs', 'unemployment','MSME','MSMEs'],
    'budget': ['deficit', 'surplus', 'expenditure'],
    'education': ['education', 'schools', 'youth','children','primary education','secondary education'],
    'defence': ['defence', 'national', 'security','threat'],
    'poverty' : ['poverty','poverty line','poor'],
    'women' :['women','women empowerment','women led','women participation','girl child']
}


In [ ]:
import re

# Function to count keyword occurrences for a specific theme
def count_keywords(speech, keywords):
    count = 0
    for keyword in keywords:
        count += len(re.findall(r'\b' + re.escape(keyword) + r'\b', speech.lower()))
    return count

# Add columns for each theme, counting the occurrences of theme-related words
for theme, keywords in themes.items():
    df_grouped[theme] = df_grouped['Speech'].apply(lambda x: count_keywords(x, keywords))

# Display the dataframe with theme counts
df_grouped.head()
df_grouped.info()
df_grouped.to_csv('speech_with_theme_counts.csv', index=False)


In [ ]:
import plotly.express as px

# Group by Finance Minister and sum the keyword counts for each theme
theme_counts_by_minister = df_grouped.groupby('Minister')[list(themes.keys())].sum().reset_index()

# Melt the DataFrame to make it long-form for easier plotting
theme_counts_melted = theme_counts_by_minister.melt(id_vars='Minister', var_name='Theme', value_name='Count')

# Plot the theme mentions using Plotly Express
fig = px.bar(theme_counts_melted,
             x='Minister',
             y='Count',
             color='Theme',
              color_discrete_sequence=px.colors.qualitative.T10,
             title='Theme Mentions by Finance Minister',
             labels={'Count': 'Number of Mentions', 'Minister': 'Finance Minister'},
             barmode='group',
             template='plotly_dark')

# Customize layout
fig.update_layout(xaxis_title='Finance Minister', yaxis_title='Number of Mentions', height=600, width=1000)
fig.show()


In [ ]:
import plotly.express as px

# Plot the theme mentions using a stacked bar chart
fig = px.bar(theme_counts_melted,
             x='Minister',
             y='Count',
             color='Theme',
             title='Comparison of Themes by Finance Minister',
             labels={'Count': 'Number of Mentions', 'Name of Finance Minister': 'Finance Minister'},
             barmode='stack',
             template='plotly_dark') # Use 'stack' to create a stacked bar chart

fig.update_layout(xaxis_title='Finance Minister', yaxis_title='Number of Mentions', height=600, width=1000)
fig.show()


# LDA

In [ ]:
!pip install pandas spacy gensim pyLDAvis


In [ ]:
import spacy
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
from gensim.models import LdaModel

# Load spaCy model for lemmatization
nlp = spacy.load('en_core_web_sm')

# Function to preprocess text (tokenization, lemmatization, stopwords removal)
def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return tokens

# Define custom stopwords (domain-specific words)
custom_stopwords = ['cent', 'crore', 'propose', 'year', 'sector', 'government', 'duty', 'lakh','rate','shall','new','high','provide','rs']

# Load the spaCy model and extend its stopwords
nlp = spacy.load('en_core_web_sm')
for word in custom_stopwords:
    nlp.vocab[word].is_stop = True

# Apply preprocessing to each speech
df_grouped['tokens'] = df_grouped['Speech'].apply(preprocess)

# Create a Gensim dictionary and corpus
dictionary = Dictionary(df_grouped['tokens'])
corpus = [dictionary.doc2bow(text) for text in df_grouped['tokens']]

# Display the processed tokens and corpus
print(dictionary)
print(corpus[:1])  # Display the first document's bag of words


In [ ]:
# Set the number of topics (you can adjust this based on the results)
num_topics = 5

# Train the LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, random_state=42)

# Print the topics with words
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Prepare the pyLDAvis visualization
lda_display = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

# Display the visualization in
pyLDAvis.display(lda_display)


#Sentiment Analysis

In [ ]:
!pip install vaderSentiment


In [ ]:
# Load the dataset (replace 'path_to_your_budget_file.csv' with your actual file path)
df_senti = pd.read_csv('/content/Speech_20years.csv')

# Clean the column names if needed
#df_senti.columns = df.columns.str.strip()
# Replace NaN values in the 'Speech' column with an empty string
df_senti['Speech'] = df['Speech'].fillna('')


# Display the first few rows of the dataframe
df_senti.head()

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Replace NaN values in the 'Speech' column with an empty string
df['Speech'] = df['Speech'].fillna('')

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to calculate sentiment for each speech
def get_sentiment(text):
    return analyzer.polarity_scores(text)

# sentiment analysis to each speech
df_senti['Sentiment'] = df['Speech'].apply(lambda speech: get_sentiment(speech))

# Split the VADER sentiment into individual columns
df_sentiment = df_senti['Sentiment'].apply(pd.Series)

# Merge the sentiment scores back into the original dataframe
df = pd.concat([df_senti, df_sentiment], axis=1)

# Display the updated dataframe with sentiment scores
df.head()
df.tail()
df['Year']


In [ ]:
# Extract the starting year from the 'Year' column
df['Year'] = df['Year'].astype(str).str.split('-').str[0]

# Convert the 'Year' column to numeric
df['Year'] = df['Year'].astype(str).str.split('-').str[0]
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')



In [ ]:
# Calculate the average sentiment score for each year
sentiment_over_time = df.groupby('Year')['compound'].mean().reset_index()

# Ensure the 'Year' column is sorted numerically
sentiment_over_time = sentiment_over_time.sort_values('Year')

# Plot the line chart for sentiment over time
import plotly.express as px

fig = px.line(sentiment_over_time,
              x='Year',
              y='compound',
              title='Sentiment of Budget Speeches Over Time',
              labels={'compound': 'Sentiment Score', 'Year': 'Year'},
              template='plotly_dark')

fig.update_layout(xaxis_title='Year', yaxis_title='Average Sentiment Score')
fig.show()


In [ ]:
# Calculate the average sentiment score for each finance minister
sentiment_by_minister = df.groupby('Minister')['compound'].mean().reset_index()

fig = px.bar(sentiment_by_minister,
             x='Minister',
             y='compound',
             title='Average Sentiment by Finance Minister',
             labels={'compound': 'Sentiment Score', 'Minister': 'Finance Minister'},
            template='plotly_dark')

fig.update_layout(xaxis_title='Finance Minister', yaxis_title='Average Sentiment Score')
fig.show()


# NER VISULISATION

In [ ]:

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

# Load your budget speeches dataset
df = pd.read_csv('/content/Speech_20years.csv')
df['Year'] = df['Year'].astype(str).str.split('-').str[0]
df['Speech'] = df['Speech'].fillna('')
# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]  # Extract entity text and label
    return entities

# Apply the function to extract entities
df['Entities'] = df['Speech'].apply(extract_entities)

# Create a dataframe of entity mentions per speech
entity_data = []
for index, row in df.iterrows():
    for entity, label in row['Entities']:
        entity_data.append({'Year': row['Year'], 'Entity': entity, 'Label': label})

entity_df = pd.DataFrame(entity_data)


In [ ]:
# Count the frequency of each entity label over time
entity_counts = entity_df.groupby(['Year', 'Label']).size().reset_index(name='Count')

# Optional: Filter for the top N most common entity types (e.g., ORG, GPE, DATE)
top_entity_labels = entity_counts.groupby('Label')['Count'].sum().nlargest(5).index
entity_counts = entity_counts[entity_counts['Label'].isin(top_entity_labels)]


In [ ]:

# Create a line chart to show how entity mentions changed over time
fig = px.line(entity_counts,
              x='Year',
              y='Count',
              color='Label',
              title='Entity Mentions Over Time',
              labels={'Count': 'Number of Mentions', 'Year': 'Year', 'Label': 'Entity Type'})

# Customize the chart
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Mentions',
    template='plotly_dark',
    paper_bgcolor='#2B2B2B',
    plot_bgcolor='#2B2B2B',
    font_color='white'
)

fig.show()


In [ ]:
# Function to extract organizations (ORG) from speeches
def extract_orgs(text):
    doc = nlp(text)
    orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']  # Extract only organizations (ORG)
    return orgs

# Apply the function to extract organizations from each speech
df['Organizations'] = df['Speech'].apply(extract_orgs)

# Create a dataframe of organization mentions per speech
org_data = []
for index, row in df.iterrows():
    for org in row['Organizations']:
        org_data.append({'Year': row['Year'], 'Organization': org})

org_df = pd.DataFrame(org_data)

# Count the number of times each organization is mentioned per year
org_counts = org_df.groupby(['Year', 'Organization']).size().reset_index(name='Count')

# Optional: Filter for the top N organizations mentioned most frequently
top_orgs = org_counts.groupby('Organization')['Count'].sum().nlargest(10).index
org_counts = org_counts[org_counts['Organization'].isin(top_orgs)]


In [ ]:
import plotly.express as px

# Create a bar chart showing how many times each organization is mentioned over time
fig = px.bar(org_counts,
             x='Year',
             y='Count',
             color='Organization',
             title='Organization Mentions Over Time',
             labels={'Count': 'Number of Mentions', 'Year': 'Year', 'Organization': 'Organization'},
             barmode='group')

# Customize the chart (dark theme)
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Mentions',
    template='plotly_dark',
    paper_bgcolor='#2B2B2B',
    plot_bgcolor='#2B2B2B',
    font_color='white'
)

fig.show()


# Shifting policies over time

In [ ]:
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher

# Load your budget speeches dataset (replace with your file path)
df = pd.read_csv('/content/Speech_20years.csv')
df['Speech'] = df['Speech'].fillna('')

# Clean column names (if necessary)
df.columns = df.columns.str.strip()

# Ensure year is properly formatted
df['Year'] = df['Year'].astype(str).str.split('-').str[0]
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize PhraseMatcher and define patterns for organization names
matcher = PhraseMatcher(nlp.vocab)

# Define the list of organizations you want to match
organization_list = ["Swachh Bharat", "Digital India", "Make in India", "National Rural Employment Guarantee Scheme",
                     "Smart Cities", "Sarva Siksha Abhiyan", "Viksit Bharat","Pradhan Mantri Jan Dhan Yojana","Atmanirbhar Bharat",
                     "Pradhan Mantri Awas Yojana- Grami","PM GatiShakti"]

# Create phrase patterns for the organizations
patterns = [nlp(org) for org in organization_list]
matcher.add("ORG", patterns)

# Function to extract matched organizations from the speech
def extract_custom_entities(text):
    doc = nlp(text)
    matches = matcher(doc)
    orgs = [doc[start:end].text for match_id, start, end in matches]
    return orgs

# Apply the custom entity extraction function to each speech
df['Organizations'] = df['Speech'].apply(extract_custom_entities)

# Check results
df[['Speech', 'Organizations']].head()


In [ ]:
# Calculate organization frequencies by year
org_frequency = pd.DataFrame()

for org in organization_list:
    df[org + '_count'] = df['Organizations'].apply(lambda x: x.count(org))
    yearly_count = df.groupby('Year')[org + '_count'].sum().reset_index()
    yearly_count['Organization'] = org
    org_frequency = pd.concat([org_frequency, yearly_count])

# Melt the DataFrame for visualization
org_frequency_melted = pd.melt(org_frequency,
                               id_vars=['Year'],
                               value_vars=[col for col in df.columns if col.endswith('_count')],
                               var_name='Organization',
                               value_name='count')

# Visualize the frequency of organizations over time
fig = px.bar(org_frequency_melted,
              x='Year',
              y='count',
              color='Organization',
              title='Frequency of Specific Policies Over Time',
              labels={'count': 'Frequency', 'Year': 'Year'},
              template='plotly_dark')

fig.update_layout(
    annotations=[
        dict(
            text="The frequency of specific policy mentions over time in budget speeches.",
            xref="paper", yref="paper",
            x=0, y=1.09,  # Position of subtitle
            showarrow=False,
            font=dict(size=14, color="white")  # Font size and color for the subtitle
        )
    ])

fig.show()


In [ ]:
# Load spaCy model for NER

nlp = spacy.load("en_core_web_sm")

# Load your budget speeches dataset (replace with your file path)
df = pd.read_csv('/content/Speech_20years.csv')
df['Speech'] = df['Speech'].fillna('')

# Clean column names (if necessary)
df.columns = df.columns.str.strip()

# Ensure year is properly formatted
df['Year'] = df['Year'].astype(str).str.split('-').str[0]
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Function to extract organization and person entities
def extract_entities(text, org_list, person_list):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if (ent.label_ == "ORG" and ent.text in org_list) or (ent.label_ == "PERSON" and ent.text in person_list)]
    return entities

# Define the list of organization names you want to track
organization_list = ["Swachh Bharat", "Digital India", "Make in India", "National Rural Employment Guarantee Scheme",
                     "Smart Cities","Sarva Siksha Abhiyan","Viksit Bharat"]

# Define the list of person names you want to track
person_list = ["Swachh Bharat", "Digital India", "Make in India", "National Rural Employment Guarantee Scheme",
                     "Smart Cities","Sarva Siksha Abhiyan","Viksit Bharat"]

# Apply the function to extract these organizations and persons for each speech
df['Entities'] = df['Speech'].apply(lambda text: extract_entities(text, organization_list, person_list))

# Display the first few rows of the DataFrame
df.head()



In [ ]:
# Function to count occurrences of specific entities (organizations or persons)
def count_entities(entities, target_entity):
    return entities.count(target_entity)

# Create a new DataFrame to store entity (organization/person) frequencies over time
entity_frequency = pd.DataFrame()

# Loop through each organization and person, and calculate their frequency by year
for entity in organization_list + person_list:
    df[entity + '_count'] = df['Entities'].apply(lambda x: count_entities(x, entity))
    yearly_count = df.groupby('Year')[entity + '_count'].sum().reset_index()
    yearly_count['Entity'] = entity
    entity_frequency = pd.concat([entity_frequency, yearly_count])

# Display the resulting DataFrame
entity_frequency.tail()